# Saving SCF results on disk and SCF checkpoints

For longer DFT calculations it is pretty standard to run them on a cluster
in advance and to perform postprocessing (band structure calculation,
plotting of density, etc.) at a later point and potentially on a different
machine.

To support such workflows DFTK offers the two functions `save_scfres`
and `load_scfres`, which allow to save the data structure returned
by `self_consistent_field` on disk or retrieve it back into memory,
respectively. For this purpose DFTK uses the
[JLD2.jl](https://github.com/JuliaIO/JLD2.jl) file format and Julia package.

!!! note "Availability of `load_scfres`, `save_scfres` and checkpointing"
    As JLD2 is an optional dependency of DFTK these three functions are only
    available once one has *both* imported DFTK and JLD2 (`using DFTK`
    and `using JLD2`).

!!! warning "DFTK data formats are not yet fully matured"
    The data format in which DFTK saves data as well as the general interface
    of the `load_scfres` and `save_scfres` pair of functions
    are not yet fully matured. If you use the functions or the produced files
    expect that you need to adapt your routines in the future even with patch
    version bumps.

To illustrate the use of the functions in practice we will compute
the total energy of the O₂ molecule at PBE level. To get the triplet
ground state we use a collinear spin polarisation
(see Collinear spin and magnetic systems for details)
and a bit of temperature to ease convergence:

In [1]:
using DFTK
using LinearAlgebra
using JLD2

d = 2.079  # oxygen-oxygen bondlength
a = 9.0    # size of the simulation box
lattice = a * I(3)
O = ElementPsp(:O; psp=load_psp("hgh/pbe/O-q6.hgh"))
atoms     = [O, O]
positions = d / 2a * [[0, 0, 1], [0, 0, -1]]
magnetic_moments = [1., 1.]

Ecut  = 10  # Far too small to be converged
model = model_PBE(lattice, atoms, positions; temperature=0.02, smearing=Smearing.Gaussian(),
                  magnetic_moments)
basis = PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])

scfres = self_consistent_field(basis, tol=1e-2, ρ=guess_density(basis, magnetic_moments))
save_scfres("scfres.jld2", scfres);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ------
  1   -27.64599194796                   -0.13    0.001    6.0    126ms
  2   -28.92297797871        0.11       -0.82    0.677    2.0    379ms
  3   -28.93104759036       -2.09       -1.14    1.179    2.5   93.4ms
  4   -28.93770137867       -2.18       -1.19    1.771    2.0   80.4ms
  5   -28.93945452679       -2.76       -1.25    2.000    2.0    117ms
  6   -28.93959141462       -3.86       -1.91    1.976    1.0   68.4ms
  7   -28.93961204796       -4.69       -3.16    1.985    1.0   69.1ms


In [2]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             16.7723631
    AtomicLocal         -58.4963306
    AtomicNonlocal      4.7097669 
    Ewald               -4.8994689
    PspCorrection       0.0044178 
    Hartree             19.3618658
    Xc                  -6.3913631
    Entropy             -0.0008630

    total               -28.939612047963

The `scfres.jld2` file could now be transferred to a different computer,
Where one could fire up a REPL to inspect the results of the above
calculation:

In [3]:
using DFTK
using JLD2
loaded = load_scfres("scfres.jld2")
propertynames(loaded)

(:α, :history_Δρ, :converged, :occupation, :occupation_threshold, :algorithm, :basis, :runtime_ns, :n_iter, :history_Etot, :εF, :energies, :ρ, :n_bands_converge, :eigenvalues, :ψ, :ham)

In [4]:
loaded.energies

Energy breakdown (in Ha):
    Kinetic             16.7723631
    AtomicLocal         -58.4963306
    AtomicNonlocal      4.7097669 
    Ewald               -4.8994689
    PspCorrection       0.0044178 
    Hartree             19.3618658
    Xc                  -6.3913631
    Entropy             -0.0008630

    total               -28.939612047963

Since the loaded data contains exactly the same data as the `scfres` returned by the
SCF calculation one could use it to plot a band structure, e.g.
`plot_bandstructure(load_scfres("scfres.jld2"))` directly from the stored data.

Notice that both `load_scfres` and `save_scfres` work by transferring all data
to/from the master process, which performs the IO operations without parallelisation.
Since this can become slow, both functions support optional arguments to speed up
the processing. An overview:
- `save_scfres("scfres.jld2", scfres; save_ψ=false)` avoids saving
  the Bloch wave, which is usually faster and saves storage space.
- `load_scfres("scfres.jld2", basis)` avoids reconstructing the basis from the file,
  but uses the passed basis instead. This save the time of constructing the basis
  twice and allows to specify parallelisation options (via the passed basis). Usually
  this is useful for continuing a calculation on a supercomputer or cluster.

See also the discussion on Input and output formats on JLD2 files.

## Checkpointing of SCF calculations
A related feature, which is very useful especially for longer calculations with DFTK
is automatic checkpointing, where the state of the SCF is periodically written to disk.
The advantage is that in case the calculation errors or gets aborted due
to overrunning the walltime limit one does not need to start from scratch,
but can continue the calculation from the last checkpoint.

The easiest way to enable checkpointing is to use the `kwargs_scf_checkpoints`
function, which does two things. (1) It sets up checkpointing using the
`ScfSaveCheckpoints` callback and (2) if a checkpoint file is detected,
the stored density is used to continue the calculation instead of the usual
atomic-orbital based guess. In practice this is done by modifying the keyword arguments
passed to # `self_consistent_field` appropriately, e.g. by using the density
or orbitals from the checkpoint file. For example:

In [5]:
checkpointargs = kwargs_scf_checkpoints(basis; ρ=guess_density(basis, magnetic_moments))
scfres = self_consistent_field(basis; tol=1e-2, checkpointargs...)

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   α      Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ----   ------
  1   -27.64470679864                   -0.13    0.001   0.80    6.0    236ms
  2   -28.92279098472        0.11       -0.82    0.679   0.80    2.0    675ms
  3   -28.93110627690       -2.08       -1.14    1.185   0.80    2.5   86.4ms
  4   -28.93773432204       -2.18       -1.19    1.774   0.80    2.0    105ms
  5   -28.93947922928       -2.76       -1.33    1.999   0.80    1.5   74.3ms
  6   -28.93959631314       -3.93       -1.97    1.977   0.80    1.0   69.1ms
  7   -28.93961179696       -4.81       -3.14    1.985   0.80    1.0    104ms


(ham = Hamiltonian(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :collinear), Ecut = 10.0 Ha, kgrid = MonkhorstPack([1, 1, 1])), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :collinear), Ecut = 10.0 Ha, kgrid = MonkhorstPack([1, 1, 1])), KPoint([     0,      0,      0], spin = 1, num. G vectors =  1141), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :collinear), Ecut = 10.0 Ha, kgrid = MonkhorstPack([1, 1, 1])), KPoint([     0,      0,      0], spin = 1, num. G vectors =  1141), [0.0, 0.24369393582936685, 0.9747757433174674, 2.1932454224643014, 3.8991029732698697, 6.092348395734172, 8.772981689857206, 8.772981689857206, 6.092348395734172, 3.8991029732698697  …  2.680633294123035, 4.386490844928604, 6.5797362673929065, 9.260369561515938, 9.260369561515938, 6.5797362673929065, 4.386490844928604, 2.680633294123035, 1.46

Notice that the `ρ` argument is now passed to kwargs_scf_checkpoints instead.
If we run in the same folder the SCF again (here using a tighter tolerance),
the calculation just continues.

In [6]:
checkpointargs = kwargs_scf_checkpoints(basis; ρ=guess_density(basis, magnetic_moments))
scfres = self_consistent_field(basis; tol=1e-3, checkpointargs...)

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   α      Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ----   ------
  1   -28.93961286214                   -3.34    1.985   0.80   12.0    179ms


(ham = Hamiltonian(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :collinear), Ecut = 10.0 Ha, kgrid = MonkhorstPack([1, 1, 1])), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :collinear), Ecut = 10.0 Ha, kgrid = MonkhorstPack([1, 1, 1])), KPoint([     0,      0,      0], spin = 1, num. G vectors =  1141), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(gga_x_pbe+gga_c_pbe, spin_polarization = :collinear), Ecut = 10.0 Ha, kgrid = MonkhorstPack([1, 1, 1])), KPoint([     0,      0,      0], spin = 1, num. G vectors =  1141), [0.0, 0.24369393582936685, 0.9747757433174674, 2.1932454224643014, 3.8991029732698697, 6.092348395734172, 8.772981689857206, 8.772981689857206, 6.092348395734172, 3.8991029732698697  …  2.680633294123035, 4.386490844928604, 6.5797362673929065, 9.260369561515938, 9.260369561515938, 6.5797362673929065, 4.386490844928604, 2.680633294123035, 1.46

Since only the density is stored in a checkpoint
(and not the Bloch waves), the first step needs a slightly elevated number
of diagonalizations. Notice, that reconstructing the `checkpointargs` in this second
call is important as the `checkpointargs` now contain different data,
such that the SCF continues from the checkpoint.
By default checkpoint is saved in the file `dftk_scf_checkpoint.jld2`, which can be changed
using the `filename` keyword argument of `kwargs_scf_checkpoints`. Note that the
file is not deleted by DFTK, so it is your responsibility to clean it up. Further note
that warnings or errors will arise if you try to use a checkpoint, which is incompatible
with your calculation.

We can also inspect the checkpoint file manually using the `load_scfres` function
and use it manually to continue the calculation:

In [7]:
oldstate = load_scfres("dftk_scf_checkpoint.jld2")
scfres   = self_consistent_field(oldstate.basis, ρ=oldstate.ρ, ψ=oldstate.ψ, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ------
  1   -28.93868505229                   -2.27    1.986    5.0   81.9ms
  2   -28.93948401220       -3.10       -2.63    1.986    1.0   95.5ms
  3   -28.93960560229       -3.92       -2.52    1.985    3.5    111ms
  4   -28.93960794555       -5.63       -2.63    1.985    1.0   68.2ms
  5   -28.93961115461       -5.49       -2.84    1.985    1.0    104ms
  6   -28.93961142548       -6.57       -2.84    1.985    1.0   61.5ms
  7   -28.93961200061       -6.24       -2.87    1.985    1.0   61.6ms
  8   -28.93961248510       -6.31       -2.93    1.985    1.0   85.9ms
  9   -28.93961286234       -6.42       -3.03    1.985    1.0   67.2ms
 10   -28.93961289989       -7.43       -3.15    1.985    1.0   67.2ms
 11   -28.93961297529       -7.12       -3.22    1.985    1.0   78.1ms
 12   -28.93961297782       -8.60       -3.21    1.985    1.0   69.1ms
 13   -

Some details on what happens under the hood in this mechanism: When using the
`kwargs_scf_checkpoints` function, the `ScfSaveCheckpoints` callback is employed
during the SCF, which causes the density to be stored to the JLD2 file in every iteration.
When reading the file, the `kwargs_scf_checkpoints` transparently patches away the `ψ`
and `ρ` keyword arguments and replaces them by the data obtained from the file.
For more details on using callbacks with DFTK's `self_consistent_field` function
see Monitoring self-consistent field calculations.

(Cleanup files generated by this notebook)

In [8]:
rm("dftk_scf_checkpoint.jld2")
rm("scfres.jld2")